In [ ]:
import pandas as p
import oandapy
co = p.read_csv('/mldev/bin/datafeeds/config.csv', header=None)
co

In [ ]:
import pandas as p
import oandapy
co = p.read_csv('/mldev/bin/datafeeds/config.csv', header=None)
loginIndex = 1
env0=co.ix[loginIndex,1]
access_token0=co.ix[loginIndex,2]
oanda0 = oandapy.API(environment=env0, access_token=access_token0)

accid = 5545219
acco = oanda0.get_account(accid)
maccount = p.DataFrame(acco, index=[0])
print maccount.transpose()

In [ ]:
accounts.Accounts(accountID="101-004-1984564-001")

In [ ]:
from exampleauth import exampleAuth
accountID, access_token = exampleAuth()

In [ ]:
# -*- coding: utf-8 -*-
#res = accounts.AccountList()
#req = client.request(res)
#df = p.DataFrame(req['accounts'])
#print df

accountID="101-004-1984564-001"

#res = accounts.AccountDetails?
#(accountID)

access_token = "f55f8626416a19d784732a4a19ec69d0-2828876dabe1996cfedfa96f50e6d36b"
accountID = "101-004-1984564-001"

import oandapyV20
#import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.trades as trades
from oandapyV20.endpoints.pricing import PricingStream
from oandapyV20.endpoints.accounts import AccountSummary

#client = API(access_token=access_token, headers={"Content-Type": "application/json"})
#client = oandapyV20.API(access_token=access_token)
client = oandapyV20.API(access_token=access_token, environment="practice")
#client = oandapyV20.API(access_token=access_token)
#client.api_url = 'https://test.com'
print client.environment

try:
    instruments = "DE30_EUR,EUR_USD,EUR_JPY"
    r = PricingStream(accountID=accountID, params={"instruments":instruments})    
    print r
    #print r.ENDPOINT
    print r.METHOD
    print r.status_code
    print r.response
    #print client.request(r)
    rv = client.request(r)
    for ri in rv:
        ri
except Exception as e:
    print e
try:
    r = AccountSummary(accountID="101-004-1984564-001")
    print r
    print r.METHOD
    print r.status_code
    print r.response
    rv = client.request(r)
    print rv
except Exception as e:
    print e
try:
    r = AccountSummary(accountID="101-004-1984564-001")
    print r
    print r.METHOD
    print r.status_code
    print r.response
    rv = client.request(r)
    print rv
except Exception as e:
    print e
try:
    r = trades.TradesList(accountID=accountID)
    print r
    print r.METHOD
    print r.status_code
    print r.response
    rv = client.request(r)
    print rv
except Exception as e:
    print e
try:
    r = accounts.AccountList()
    print r
    print r.METHOD
    print r.status_code
    print r.response
    rv = client.request(r)
    print rv
except Exception as e:
    print e

#res = accounts.AccountSummary(accountID)
#print res
#print res.status_code
#client.request(res)
#client.request?

In [ ]:
from oandapyV20.exceptions import V20Error
import oandapyV20.endpoints.pricing as pricing
# list of requests
lor = []
# request trades list
res = trades.TradesList(accountID)
print res
print res.ENDPOINT
lor.append(res)
# request accounts list
lor.append(accounts.AccountList())
# request pricing info
params={"instruments": "DE30_EUR,EUR_GBP"}
lor.append(pricing.PricingInfo(accountID, params=params))

for r in lor:
    try:
        rv = client.request(r)
        # put request and response in 1 JSON structure
        print("{}".format(json.dumps({"request": "{}".format(r),
                                      "response": rv}, indent=2)))
    except V20Error as e:
        print("OOPS: {:d} {:s}".format(e.code, e.msg))

In [ ]:
res = accounts.AccountDetails

In [ ]:
res = accounts.AccountSummary

In [ ]:
res = accounts.AccountSummary

In [ ]:
res = accounts.AccountSummary

In [ ]:
res = accounts.AccountSummary
res

In [ ]:
import json
import pandas as p
from oandapyV20 import API    # the client
import oandapyV20.endpoints.trades as trades
import oandapyV20.endpoints.accounts as accounts

access_token = "f55f8626416a19d784732a4a19ec69d0-2828876dabe1996cfedfa96f50e6d36b"
accountID = "101-004-1984564-002"
client = API(access_token=access_token)

al = accounts.AccountList()
ral = client.request(al)
ral
df = p.DataFrame(ral['accounts'])
print df

print
print df.ix[0,'id']

# request trades list
try:
    r = trades.TradesList(accountID)
    #print r
    rv = client.request(r)
    print("RESPONSE:\n{}".format(json.dumps(rv, indent=2)))
except Exception as e:
    print e

In [ ]:
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.transactions as transactions
import oandapyV20.endpoints.accounts as accounts

#transactions.TransactionList?
instruments

al = accounts.AccountList()
print al
ral = client.request(al)
ral
df = p.DataFrame(ral['accounts'])
print df

In [ ]:
import v20
api = v20.Context(hostname='api-fxpractice.oanda.com', port=443, ssl=True, token='e6ab562b039325f12a026c6fdb7b71bb-b3d8721445817159410f01514acd19hbc')
#streaming_hostname: stream-fxpracatice.oanda.com
#username: user
#accounts:
#- 101-001-100000-001
#- 101-001-100000-002
#active_account: 101-001-100000-001
api.primitives.

In [ ]:
import numpy as n
print p.DataFrame(n.power(2, n.array(range(7+1))))

In [ ]:
#source: https://www.youtube.com/watch?v=DIIUNtUVDPI
import pandas as p
import numpy as n
#import matplotlib.pylab as plt

%pylab inline

df = p.DataFrame([1,2,3,4])
df[1] = n.power(df[0],2)
df[2] = n.power(df[0],3)
df[3] = n.power(df[0],n.e)
print df
df.plot()
#plt.show()

In [ ]:
%reload_ext autoreload
%autoreload 2
import sys
def defp(pt):
    try:    sys.path.index(pt)
    except: sys.path.append(pt)
defp('/ml.dev/bin')
defp('/ml.dev/lib/oanda/oandapy')
from qoreliquid import getSyntheticCurrencyTable
from oandaq import OandaQ
import pandas as p
import numpy as n
#oq = OandaQ(selectOandaAccount=0, selectOandaSubAccount=6)
#df = getSyntheticCurrencyTable(oq.oanda2, oq, ['EUR_GBP','AUD_USD','NZD_USD'])#.transpose()
#print df.ix[:,'pairedCurrencyAsk pairedCurrencyBid'.split(' ')]
#print df.ix[:,'pairedCurrencyAsk pairedCurrencyBid'.split(' ')]
#print df

In [ ]:
import sys
def defp(pt):
    try:    sys.path.index(pt)
    except: sys.path.append(pt)
defp('/mldev/lib/bitcoin/poloniex/thiagof_py-poloniex.github.py.git')
%pylab inline
from poloniex import Poloniex
from datetime import datetime
from datetime import timedelta
from qoreliquid import normalizeme
from qoreliquid import sigmoidme
pol = Poloniex('M8YTJIKE-2EIE8VV2-7UP6Z9O0-PJNGRPV4', '7ed5f13cee6469c3790f236c28d9b9dcd3b1714f9b5a310c8f25f221348ec5c2d0149cda35d58f53a34cae20d8e246f8462d7e95abcaa2bac4062ad24fb0034d')

In [ ]:
#pol.returnChartData()
start = datetime.now() - timedelta(hours=24)
df = {}
#bss = ['BTC_ETH', 'BTC_DOGE']
bss = list(p.DataFrame(pol.returnBalances().keys()).sort_values(by=0).transpose().get_values()[0])
bss = map(lambda x: 'BTC_%s'%x, bss)
#print bss
mdf=p.DataFrame([])
for i in bss[0:6]:
    print '--- %s' % i
    try:
        res = pol.returnChartData(i, 300, start)
        df[i] = p.DataFrame(res).sort_values(by='date').set_index('date')
        #with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000): print df[i].tail(15)
        mdf[i] = df[i].ix[:,'close']
    except Exception as e:
        #print e
        #print res['error']
        ''
#df.ix[:,'close'].plot()
mdf = mdf.ffill().bfill()
mdf = normalizeme(mdf)
mdf = sigmoidme(mdf)
print mdf
mdf.plot()

In [ ]:
mdf#.ix[:,0:6]#.plot(legend=0)`\

In [ ]:
#start = datetime.now() - timedelta(hours=1)
#res = pol.returnTradeHistory('BTC_DOGE', start)
#df=p.DataFrame(res)
#df=df.convert_objects(convert_numeric=True)
#print df.dtypes
gdf=df.groupby('type')
gdfsum=gdf.sum()
#gdfsum gdfsum.transpose()1
gdfsum.ix['total', 'total'] = - gdfsum.ix['buy','total'] + gdfsum.ix['sell','total']
print gdfsum
print gdfsum.ix['buy','total']
print gdfsum.ix['sell','total']


In [ ]:
import sys
def defp(pt):
    try:    sys.path.index(pt)
    except: sys.path.append(pt)
defp('/ml.dev/bin')
defp('/ml.dev/lib/oanda/oandapy')
from qoreliquid import normalizeme
from qoreliquid import sigmoidme
import pandas as p
import oandapy
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 20, 5

co = p.read_csv('/mldev/bin/datafeeds/config.csv', header=None)

co.ix[1,4] = 'oanda1'
env1=co.ix[1,1]
access_token1=co.ix[1,2]
oanda1 = oandapy.API(environment=env1, access_token=access_token1)

co.ix[0,4] = 'oanda2'
env2=co.ix[0,1]
access_token2=co.ix[0,2]
oanda2 = oandapy.API(environment=env2, access_token=access_token2)

try:
    acc = oanda2.get_accounts()['accounts']
    #print 'acc'
    #print p.DataFrame(acc)
    accid = acc[2]['accountId']
    #print 'using account: {0}'.format(accid)
except:
    ''

In [ ]:
co

In [ ]:
import pymongo as mong
import ujson
mongo = mong.MongoClient(host='127.0.0.1', port=3311)
dfm = p.DataFrame()
for i in mongo.ql.broker_oanda_accounts.find():
    #print p.DataFrame(i, index=[0]).transpose()
    #print i['user']
    #print i['utctime']
    df = p.DataFrame(ujson.loads(i['data']))
    df['user']    = i['user']
    df['utctime'] = i['utctime']
    for j in df.index:
        df.ix[j, 'idx'] = '%s-%s-%s' % (df.ix[j, 'user'], df.ix[j, 'utctime'], j)
    df['indx'] = df.index
    df = df.set_index('idx')
    #print df
    dfm = dfm.combine_first(df)
dfm = dfm.sort_values(by=['user', 'accountId'], ascending=[True, True])#.tail(20)
###
#aa = 'balance marginAvail marginUsed realizedPl unrealizedPl openTrades'
aa = 'balance marginUsed realizedPl unrealizedPl openTrades'
bb = 'utctime %s' % aa
ffds = aa.split(' ')
#print dfm.where(dfm['accountName'] == 'sub1')
#print dfm
#print dfm[dfm['accountName'] == 'sub1']
df = dfm[dfm['accountId'] == 947325].sort_values(by=['utctime'], ascending=[True]).ix[:, bb.split(' ')]
df = normalizeme(df)
df = sigmoidme(df)
#print df
%pylab inline
#rcParams['figure.figsize'] = 20, 5
plot(df.ix[:, ffds])
legend(ffds, loc=2)
#show()

In [ ]:
#from matplotlib.pylab import plot, legend, title, show #, imshow, tight_layout
#from pylab import rcParams
%pylab inline
#rcParams['figure.figsize'] = 20, 5
plot(df)
show()

In [ ]:
cc = [0.5, 1, 200.0/15, 2000.0/15, 2500.0/15, 50, 500, 1000, 2400, 5000]
from qorequant import QoreQuant
qq = QoreQuant(oandaInit=False, statWingInit=False)
print qq.calcCapitalRequirements(cc)
#print calcCapital(5000)